In [1]:
import numpy as np
import pandas as pd

from urllib.parse import urlparse

import mlflow
import mlflow.sklearn

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [2]:
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(experiment_name='Wine Regression')

<Experiment: artifact_location='mlflow-artifacts:/656912751409069458', creation_time=1673555786573, experiment_id='656912751409069458', last_update_time=1673555786573, lifecycle_stage='active', name='Wine Regression', tags={}>

In [3]:
tags = {"team": "Analytics Principal",
        "dataset": "Wine",
        "release.version": "2.2.2"}

In [4]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [5]:
np.random.seed(40)

In [6]:
csv_url = ("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/data/winequality-red.csv")

In [7]:
data = pd.read_csv(csv_url, sep=";")

In [8]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
train, test = train_test_split(data)

In [10]:
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [11]:
alpha = 0.8
l1_ratio = 0.6
random_state = 42

In [12]:
with mlflow.start_run() as run:
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=random_state)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_param("random_state", random_state)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    # if tracking_url_type_store != "file":
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        # mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    # else:
    #     mlflow.sklearn.log_model(lr, "model")
    
    
    mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")

Elasticnet model (alpha=0.800000, l1_ratio=0.600000):
  RMSE: 0.8326350511178233
  MAE: 0.6676630843299566
  R2: 0.01770234373563795


/opt/anaconda3/envs/mlflow-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...
2023/01/12 19:29:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetWineModel, version 2
Created version '2' of model 'ElasticnetWineModel'.
